In [1]:
!pip install pyforest

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyforest: filename=pyforest-1.1.2-py2.py3-none-any.whl size=15928 sha256=8563c20c059abc11d368488adba11eff800259600d8e85f74025e8966dbbda7e
  Stored in directory: c:\users\kumar\appdata\local\pip\cache\wheels\66\4f\3c\cbdf5336b5a84f39cb6e3d76eecb9acf3f99a2c072785afecc
Successfully built pyforest


In [5]:
!pip install pycaw

In [5]:
!pip install mediapipe

  Using cached mediapipe-0.10.14-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached jax-0.4.34-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.4.34-cp312-cp312-win_amd64.whl.metadata (1.0 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.5.1-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached ml_dtypes-0.5.0-cp312-cp312-win_amd64.whl.metadata (22 kB)
Using cached mediapipe-0.10.14-cp312-cp312-win_amd64.whl (50.8 MB)
Using cached sounddevice-0.5.1-py3-none-win_amd64.whl (363 kB)
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
Using cached jax-0.4.34-py3-none-any.whl (2.1 MB)
Using cached jaxlib-0.4.34-cp312-cp312-win_amd64.whl (55.3 MB)
Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl (45.5 MB)
Using cached ml_dtypes-0.5.0-cp312-cp312-win_amd64.whl (213 kB)


In [1]:
!pip install comtypes

   ---------------------------------------- 0.0/229.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/229.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/229.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/229.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/229.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/229.4 kB ? eta -:--:--
   ------ -------------------------------- 41.0/229.4 kB 196.9 kB/s eta 0:00:01
   ------------- ------------------------- 81.9/229.4 kB 353.1 kB/s eta 0:00:01
   -------------------------------------  225.3/229.4 kB 808.4 kB/s eta 0:00:01
   -------------------------------------  225.3/229.4 kB 808.4 kB/s eta 0:00:01
   -------------------------------------  225.3/229.4 kB 808.4 kB/s eta 0:00:01
   -------------------------------------  225.3/229.4 kB 808.4 kB/s eta 0:00:01
   -------------------------------------  225.3/229.4 kB 808.4 kB/s eta 0:00:01
   --------

In [9]:
import sys
!{sys.executable} -m pip install opencv-python

In [1]:
import cv2
import mediapipe as mp
import math 
import numpy

from ctypes import cast,POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_,CLSCTX_ALL,None)
volume = cast(interface, POINTER(IAudioEndpointVolume))


volume.SetMasterVolumeLevel(-37.0,None)

capture = cv2.VideoCapture(0)

mp_draw = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

while True:
    success,img = capture.read()
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    #detecting the palm
    results = hands.process(imgRGB)
    """if palm has been detected successfully
       we loop through the hand to select 2 finger tips needed"""
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lmList = []
            for id,lm in enumerate(handLms.landmark):
                a, b, c = img.shape
                cx,cy = int(lm.x*a) , int(lm.y*b)
                lmList.append([id,cx,cy])
                
                #print(id,lm)
                #mp_draw.draw_landmarks(img,handLms,mp_hands.HAND_CONNECTIONS)
            if lmList:
                #maps the co ordinates of the thumb
                x1,y1 = lmList[4][1] , lmList[4][2]
                #maps the co ordinates the fore finger
                x2,y2 = lmList[8][1], lmList[8][2]
                #highlights the thumb tip
                cv2.circle(img,(x1,y1),15,(1,12,123),cv2.FILLED)
                #highlights the finger tip
                cv2.circle(img,(x2,y2),15,(1,12,123),cv2.FILLED)
                cv2.line(img,(x1,y1),(x2,y2),(1,12,123),4)
                z1,z2=(x1+x2)//2,(y1+y2)//2
                distance = math.hypot(x2-x1,y2-y1)
                
            volRange = volume.GetVolumeRange()
            minVol = volRange[0]
            maxVol = volRange[1]
            vol = numpy.interp(distance,[50,100],[minVol,maxVol])
            volPer = numpy.interp(distance,[50,100],[0,100])
            volBar = numpy.interp(distance,[50,300],[400,150])
            
            volume.SetMasterVolumeLevel(vol,None)
            cv2.putText(img,str(int(volPer)),(40,450),cv2.FONT_HERSHEY_COMPLEX,3,(1,3,5),3)
            cv2.rectangle(img,(50,150),(85,400),(123,213,122),3)
            cv2.rectangle(img,(50,int(volBar)),(85,400),(0,231,23),cv2.FILLED)
            
                
    cv2.imshow("Motion Sound Control",img)
    # Exit condition
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources and close windows
capture.release()
cv2.destroyAllWindows()

    
#length = 50-300 ---> vol = 0-100
#lib  -37 to 0.0, my system's range --> 0-100

#length - 50 ---> lib = -37 ---> system - 0
#length - 300 ---> lib = 0.0 ---> system - 100


C:\Users\kumar\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
